In [1]:
def get_matched_pairs(true_events, events):
    
    def get_max_intersection(intersections):
        mx = -1 #stores maximum intersection value
        r = -1 #stores event with maximum intersection
        for val in intersections.keys():
            if intersections[val]>mx:
                mx = intersections[val]
                r = val
            elif intersections[val] == mx:
                r = -1 #if equal intersection then ignore
        return r
    
    matched_pairs = list()
#     get intersection of events in true events

    strt = 0 #start of event
    event_map_trueevent = dict() #final dict for events 
    for i, e in enumerate(events):
        e_te = dict() #temporary dict for intersections
        idx = 0
        for j,te in enumerate(true_events):
            if te<strt:
                continue
            elif te>e:
                break
            else:
                e_te[j]=te-strt #get num of sentences of an event e in true event te
                strt = te #define the new starting point as the ending point of previous true event
                idx = j+1 #for final intersection
        e_te[idx]=e-strt
        event_map_trueevent[i] = get_max_intersection(e_te)
        

#     get intersection of true events in events

    strt = 0 #start of event
    trueevent_map_event = dict() #final dict for events 
    for i, te in enumerate(true_events):
        te_e = dict() #temporary dict for intersections
        idx = 0
        for j,e in enumerate(events):
            if e<strt:
                continue
            elif e>te:
                break
            else:
                te_e[j]=e-strt #get num of sentences of an event e in true event te
                strt = e #define the new starting point as the ending point of previous true event
                idx = j+1 #for final intersection
        te_e[idx]=te-strt
        trueevent_map_event[i] = get_max_intersection(te_e)
        
    #print("1 ",event_map_trueevent,"\n2: ", trueevent_map_event)
#         check for intersection in both mappings
    for key in event_map_trueevent.keys():
        te_key = event_map_trueevent[key] #get the matched true event for the event "key"
        if te_key in trueevent_map_event:
            if trueevent_map_event[te_key] == key:  #check if the true event is also matched to the same event
                matched_pairs.append((key, te_key))
    #print("\n",matched_pairs)
    return matched_pairs
    

In [2]:
def IoU(true_events, events, total_sentences):
    matched_pairs = get_matched_pairs(true_events, events)
    total_ev_sentences = total_tev_sentences = set(range(total_sentences))
    IoU = list()
    for (ev, tev) in matched_pairs:
#         get list of sentences for an event
        if ev == 0:
            ev_list = set(range(events[ev]))
        else :
            ev_list = set(range(events[ev-1], events[ev]))
        if tev == 0:
            tev_list = set(range(true_events[tev]))
        else :
            tev_list = set(range(true_events[tev-1], true_events[tev]))
        inter = min(len(ev_list.intersection(tev_list)), len(tev_list.intersection(ev_list)))
        union = len(ev_list.union(tev_list))
        total_ev_sentences = total_ev_sentences.difference(ev_list)
        total_tev_sentences = total_tev_sentences.difference(tev_list)
        IoU.append(inter/union)
    return sum(IoU)/(len(IoU))/(1+ (len(total_ev_sentences.intersection(total_tev_sentences))/total_sentences))

In [17]:
sentences_list = [66,49,12,34,33,38,23,57,38,39,52,64]

In [8]:
IoU([1,7,13,19,29,34,44,54], [1, 7, 17, 24, 48], 66)

0.4973913043478261

In [10]:
print("Character Based")
true_events = [[7,9,17,26,31,38,41,45,49,64,65], [1,7,13,19,29,34,44,54], [1,6,17,22,27,48], [2,9,11,13,24,33,48,58,60], [1,5,6,11]]
events_generated = [[5, 9, 14, 21, 27, 32, 38, 45, 50, 59, 64],
 [9, 14, 43, 53],
 [6, 18, 23, 27, 36, 43, 47],
 [7, 13, 20, 35, 40, 45, 51, 56, 59],
 [6, 10]]
Final_score = 0
for i in range(len(true_events)):
    Final_score += IoU(true_events[i], events_generated[i],sentences_list[i])
    print(IoU(true_events[i], events_generated[i], sentences_list[i]),"\n\n")
print("final_score = ", Final_score/5)

Character Based
0.5457908163265306 


0.38633461047254153 


0.6031073446327684 


0.3564072093483858 


0.6769230769230771 


final_score =  0.5137126115406607


In [22]:
print("Character Based with exclusion")
true_events = [[7,9,17,26,31,38,41,45,49,64,65], [1,7,13,19,29,34,44,54], [1,6,17,22,27,48], [2,9,11,13,24,33,48,58,60], [1,5,6,11]]
events_generated =[[9, 27, 32, 38, 45, 50, 59, 64],
 [9, 14, 53],
 [6, 18, 23, 43],
 [7, 13, 35, 51, 59],
 [6, 10]]
Final_score = 0
for i in range(len(true_events)):
    if i==1:
        continue
    Final_score += IoU(true_events[i], events_generated[i],sentences_list[i])
    print(IoU(true_events[i], events_generated[i], sentences_list[i]),"\n\n")
print("final_score = ", Final_score/4)

Character Based with exclusion
0.6273589065255731 


0.7097575757575759 


0.35378510378510375 


0.6769230769230771 


final_score =  0.5919561657478325


In [13]:
print("Location based without lemma")
true_events = [[7,9,17,26,31,38,41,45,49,64,65], [1,7,13,19,29,34,44,54], [1,6,17,22,27,48], [2,9,11,13,24,33,48,58,60], [1,5,6,11]]
events_generated = [[6, 7, 8, 10, 15, 27, 30, 33, 46, 51, 62, 65], [1, 10, 11, 15, 17, 23, 28, 30, 54], [2, 8, 16, 18, 23, 25, 48], [9, 47, 56, 60], [1, 2, 4, 11]]
Final_score = 0
for i in range(len(true_events)):
    Final_score += IoU(true_events[i], events_generated[i],sentences_list[i])
    print(IoU(true_events[i], events_generated[i], sentences_list[i]),"\n\n")
print("final_score = ", Final_score/5)

Location based without lemma
0.5655102040816327 


0.45833333333333326 


0.6652932110274193 


0.4685262185262185 


0.6813186813186815 


final_score =  0.5677963296574571


In [9]:
IoU([1,6,17,22,27,48], [1, 7, 17, 24, 48], 49)

0.8490151515151517

In [18]:
print("Location based without lemma with no split")
true_events = [[7,9,17,26,31,38,41,45,49,64,65], [1,6,17,22,27,48], [2,9,11,13,24,33,48,58,60], [1,5,6,11]]
events_generated =[[6, 7, 8, 10, 27, 33, 51, 62, 65], [2, 8, 18, 23, 48], [9, 60], [1, 2, 4, 11]]
Final_score = 0
for i in range(len(true_events)):
    Final_score += IoU(true_events[i], events_generated[i],sentences_list[i])
    print(IoU(true_events[i], events_generated[i], sentences_list[i]),"\n\n")
print("final_score = ", Final_score/4)

Location based without lemma with no split
0.42366946778711484 


0.6304166666666667 


0.52730339447607 


0.6813186813186815 


final_score =  0.5656770525621333


In [19]:
print("Location based without lemma with no split idx change")
true_events = [[7,9,17,26,31,38,41,45,49,64,65], [1,6,17,22,27,48], [2,9,11,13,24,33,48,58,60], [1,5,6,11]]
events_generated =[[5, 6, 7, 9, 26, 29, 32, 45, 50, 61, 64, 65], [1, 7, 17, 22, 24, 48], [8, 60], [0, 1, 3, 11]]
Final_score = 0
for i in range(len(true_events)):
    Final_score += IoU(true_events[i], events_generated[i],sentences_list[i])
    print(IoU(true_events[i], events_generated[i], sentences_list[i]),"\n\n")
print("final_score = ", Final_score/len(true_events))

Location based without lemma with no split idx change
0.7148689326476859 


0.8702068764568763 


0.4698614557485525 


0.65 


final_score =  0.6762343162132787


In [12]:
"Location based with Lemma"
true_events = [[7,9,17,26,31,38,41,45,49,64,65], [1,7,13,19,29,34,44,54], [1,6,17,22,27,48], [2,9,11,13,24,33,48,58,60], [1,5,6,11]]
events_generated = [[6, 7, 8, 10, 15, 27, 30, 33, 46, 51, 62, 65], [1, 10, 11, 15, 17, 23, 28, 30, 54], [2, 8, 16, 18, 23, 25, 48], [9, 47, 56, 60], [1, 2, 4, 11]]
Final_score = 0
for i in range(len(true_events)):
    Final_score += IoU(true_events[i], events_generated[i], sentences_list[i])
    print(IoU(true_events[i], events_generated[i], sentences_list[i]),"\n\n")
print("final_score = ", Final_score/5)

0.5655102040816327 


0.45833333333333326 


0.6652932110274193 


0.4685262185262185 


0.6813186813186815 


final_score =  0.5677963296574571


In [15]:
"Loaction and Time Based wo Lemma"
true_events = [[7,9,17,26,31,38,41,45,49,64,65], [1,7,13,19,29,34,44,54], [1,6,17,22,27,48], [2,9,11,13,24,33,48,58,60], [1,5,6,11]]
events_generated = [[6, 7, 8, 10, 15, 27, 30, 33, 46, 51, 62, 65], [1, 10, 11, 15, 17, 23, 28, 30, 54], [2, 8, 16, 18, 23, 25, 48], [9, 24, 47, 56, 60], [1, 2, 4, 11]]
Final_score = 0
for i in range(len(true_events)):
    Final_score += IoU(true_events[i], events_generated[i], sentences_list[i])
    print(IoU(true_events[i], events_generated[i], sentences_list[i]),"\n\n")
print("final_score = ", Final_score/5)

0.5655102040816327 


0.45833333333333326 


0.6652932110274193 


0.5755005005005005 


0.6813186813186815 


final_score =  0.5891911860523135


In [24]:
"Loaction and Time Based wo Lemma no split"
true_events = [[7,9,17,26,31,38,41,45,49,64,65], [1,7,13,19,29,34,44,54], [1,6,17,22,27,48], [2,9,11,13,24,33,48,58,60], [1,5,6,11]]
events_generated = [[6, 7, 8, 10, 27, 33, 51, 62, 65], [1, 10, 11, 15, 17, 30, 54], [2, 8, 18, 23, 48], [9, 24, 60], [1, 2, 4, 11]]
Final_score = 0
for i in range(len(true_events)):
    Final_score += IoU(true_events[i], events_generated[i], sentences_list[i])
    print(IoU(true_events[i], events_generated[i], sentences_list[i]),"\n\n")
print("final_score = ", Final_score/5)

0.42366946778711484 


0.5133117202082719 


0.6304166666666667 


0.6322281959378734 


0.6813186813186815 


final_score =  0.5761889463837216


In [16]:
"Loaction and Time Based with Lemma"
true_events = [[7,9,17,26,31,38,41,45,49,64,65], [1,7,13,19,29,34,44,54], [1,6,17,22,27,48], [2,9,11,13,24,33,48,58,60], [1,5,6,11]]
events_generated = [[6, 7, 8, 10, 15, 27, 30, 33, 46, 51, 62, 65], [1, 10, 11, 15, 17, 23, 28, 30, 54], [2, 8, 16, 18, 23, 25, 48], [9, 24, 47, 56, 60], [1, 2, 4, 11]]
Final_score = 0
for i in range(len(true_events)):
    Final_score += IoU(true_events[i], events_generated[i], sentences_list[i])
    print(IoU(true_events[i], events_generated[i], sentences_list[i]),"\n\n")
print("final_score = ", Final_score/5)

0.5655102040816327 


0.45833333333333326 


0.6652932110274193 


0.5755005005005005 


0.6813186813186815 


final_score =  0.5891911860523135


In [17]:
"Time Based wo Lemma"
true_events = [[7,9,17,26,31,38,41,45,49,64,65], [1,7,13,19,29,34,44,54], [1,6,17,22,27,48], [2,9,11,13,24,33,48,58,60], [1,5,6,11]]
events_generated = [[1, 19, 28, 31, 41, 58, 65], [5, 7, 8, 17, 18, 37, 41, 43, 54], [2, 18, 20, 22, 48], [1, 2, 9, 17, 28, 33, 46, 60], [1, 11]]
Final_score = 0
for i in range(len(true_events)):
    Final_score += IoU(true_events[i], events_generated[i], sentences_list[i])
    print(IoU(true_events[i], events_generated[i], sentences_list[i]),"\n\n")
print("final_score = ", Final_score/5)

0.4171169559253295 


0.3958375668901985 


0.6541466346153847 


0.5103267973856209 


0.6923076923076924 


final_score =  0.5339471294248452


In [18]:
"Time Based with Lemma"
true_events = [[7,9,17,26,31,38,41,45,49,64,65], [1,7,13,19,29,34,44,54], [1,6,17,22,27,48], [2,9,11,13,24,33,48,58,60], [1,5,6,11]]
events_generated = [[1, 19, 28, 31, 41, 58, 65], [5, 7, 8, 17, 18, 37, 41, 43, 54], [2, 18, 20, 22, 48], [1, 2, 9, 17, 28, 33, 46, 60], [1, 11]]
Final_score = 0
for i in range(len(true_events)):
    Final_score += IoU(true_events[i], events_generated[i], sentences_list[i])
    print(IoU(true_events[i], events_generated[i], sentences_list[i]),"\n\n")
print("final_score = ", Final_score/5)

0.4171169559253295 


0.3958375668901985 


0.6541466346153847 


0.5103267973856209 


0.6923076923076924 


final_score =  0.5339471294248452


In [3]:
sentences_list = [66,49,12,34,33,38,23,57,38,39,52,64]